# Preprocessing

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!pip install transformers

In [25]:
%cd '/content/drive/MyDrive/Colab Notebooks/NLPGroupCW'

/content/drive/.shortcut-targets-by-id/1qcQshV1qoMFlpNzFTKlWvvjZbTNz5V3t/NLPGroupCW


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split
from transformers import BertTokenizer, BertModel

from dataloaders import *
from processor import *

In [27]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

vocab_size = len(tokenizer.vocab)

max_sentence_length = tokenizer.max_model_input_sizes['bert-base-uncased']

In [29]:
def tokenize(corpus):
    return [tokenizer.tokenize(sentence) for sentence in corpus]

def to_ids(corpus):
    return [tokenizer.convert_tokens_to_ids(sentence) for sentence in corpus]

In [30]:
train_df = pd.read_csv('data/task-1/train.csv')
test_df = pd.read_csv('data/task-1/dev.csv')

training_data = train_df['original']
training_edits = train_df['edit']
test_data = test_df['original']
test_edits = test_df['edit']

training_grades = train_df['meanGrade']

edited_training = pd.Series(create_edited_sentences(training_data, training_edits))
edited_test = pd.Series(create_edited_sentences(test_data, test_edits))

In [31]:
training_tokens = tokenize(edited_training)
testing_tokens = tokenize(edited_test)

training_ids = to_ids(training_tokens)
testing_tokens = to_ids(testing_tokens)

In [32]:
print(training_tokens[100])
print(training_ids[100])

['trump', 'asked', 'du', '##ter', '##te', 'if', 'philippines', 'has', 'death', 'limit', ',', 'philippines', 'ambassador', 'says']
[8398, 2356, 4241, 3334, 2618, 2065, 5137, 2038, 2331, 5787, 1010, 5137, 6059, 2758]


In [33]:
BATCH_SIZE = 128

train = Task1Dataset(training_ids, training_grades)
train_dataset, validation_dataset = dataset_split(train)

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)

# Build Model

In [34]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [79]:
class BertGradePredictor(nn.Module):
    def __init__(self, bert_model, total_layers, hid_size, out_size, isBidir, drop):
        super().__init__()
        
        self.bert_model = bert_model

        self.isBidir = isBidir
        
        embed_size = bert.config.to_dict()['hidden_size']

        if total_layers < 3:
          drop = 0

        hid_output_size = hid_size
        if isBidir:
          hid_output_size = hid_output_size * 2
        
        self.drop = drop

        self.gru = nn.GRU(input_size=embed_size,
                          hidden_size=hid_size,
                          num_layers=total_layers,
                          bidirectional=isBidir,
                          batch_first=True,
                          dropout=drop)
        
        self.fc1 = nn.Linear(hid_output_size, out_size)
        
        
    def forward(self, x):

        isBidir = self.isBidir
        
        with torch.no_grad():
            x_embed = self.bert_model(x)
            x_embed = x_embed[0]
        
        cell, hid = self.gru(x_embed)
        hid_last = hid[-1,:,:]
        hid_snd_last = hid[-2,:,:]
        
        if isBidir:
            hid = F.dropout(torch.cat((hid_snd_last, hid_last), dim=1), self.drop)
        else:
            hid = F.dropout(hid_last, self.drop)
        
        out = self.fc1(hid)
        
        return out

In [ ]:
total_layers = 3
hid_size = 128
out_size = 1
drop = 0.3
isBidir = True

In [80]:
model = BertGradePredictor(bert_model,
                         total_layers,
                         hid_size,
                         out_size,
                         isBidir,
                         drop)

optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

model = model.to(device)
loss_fn = loss_fn.to(device)

In [91]:
bert_layers = model.named_parameters()
bert_layers = [(layer, parameter) for layer, parameter in bert_layers]
for i in range(len(bert_layers)):
    layer_p = bert_layers[i]
    layer = layer_p[0]
    p = layer_p[1]
    if "bert_model" in layer:
      p.requires_grad = False

In [92]:
def model_performance(output, target, print_output=False):
    """
    Returns SSE and MSE per batch (printing the MSE and the RMSE)
    """

    sq_error = (output - target)**2

    sse = np.sum(sq_error)
    mse = np.mean(sq_error)
    rmse = np.sqrt(mse)

    if print_output:
        print(f'| MSE: {mse:.2f} | RMSE: {rmse:.2f} |')

    return sse, mse

In [93]:
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_sse = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)
            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            sse, __ = model_performance(pred, trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_sse/no_observations, np.array(pred_all), np.array(trg_all)

In [94]:
def train(train_loader, validation_loader, model, number_epoch):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """
    print("Training model.")
    for epoch in range(1, number_epoch+1):
        model.train()
        epoch_loss = 0
        epoch_sse = 0
        no_observations = 0  # Observations used for training so far
        for batch in train_loader:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)
            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()
            predictions = model(feature).squeeze(1)
            optimizer.zero_grad()
            loss = loss_fn(predictions, target)
            sse, __ = model_performance(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse

        valid_loss, valid_mse, __, __ = eval(validation_loader, model)

        epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} |')

In [96]:
train(train_loader, validation_loader, model, 2)

Training model.
| Epoch: 01 | Train Loss: 0.33 | Train MSE: 0.33 | Train RMSE: 0.58 |         Val. Loss: 0.37 | Val. MSE: 0.37 |  Val. RMSE: 0.61 |
| Epoch: 02 | Train Loss: 0.33 | Train MSE: 0.33 | Train RMSE: 0.57 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.57 |
